In [ ]:
# Dataset
dataset = {
    'Acne': {
        'index': 1,
        'symptom': {
             'Acne or pimples': 0.94,
             'Skin rash': 0.38,
             'Abnormal appearing skin': 0.21,
             'Skin moles': 0.2,
             'Skin swelling': 0.17,
             'Skin growth': 0.13,
        }
    },'Eczema': {
        'index': 2,
        'symptom': {
            'Skin rash': 0.78,
            'Itching of skin': 0.49,
            'Abnormal appearing skin': 0.45,
            'Skin dryness, peeling, scaliness, or roughness': 0.33,
            'Skin lesion': 0.29,
            'Cough': 0.24,
        }
    },
    'Erythema multiforme': {
        'index': 3,
        'symptom': {
            'Skin rash': 0.92,
            'Fever': 0.67,
            'Itching of skin': 0.44,
            'Coryza': 0.44,
            'Skin lesion': 0.44,
            'Vomiting': 0.44,
        }
    },
    'Dermatitis due to sun exposure': {
        'index': 4,
        'symptom': {
            'Abnormal appearing skin': 0.74,
            'Skin lesion': 0.51,
            'Skin rash': 0.45,
            'Skin moles': 0.39,
            'Skin dryness, peeling, scaliness, or roughness': 0.27,
            'Irregular appearing scalp': 0.24,
        }
    }
}

In [ ]:
symptom = [
    'Abnormal appearing skin',
    'Acne or pimples',
    'Coryza',
    'Cough',
    'Fever',
    'Irregular appearing scalp',
    'Itching of skin',
    'Skin dryness, peeling, scaliness, or roughness',
    'Skin growth',
    'Skin lesion',
    'Skin moles',
    'Skin rash',
    'Skin swelling',
    'Vomiting'
]

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations, product

In [ ]:
# Generate all possible disease combinations
disease_combinations = list(combinations(['Acne', 'Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'], 3))
# Generate all possible probability combinations (0.1 to 0.9, step 0.1)
probabilities = np.arange(0.1, 1.0, 0.1)
# print(disease_combinations)
# print(probabilities)

In [ ]:
# Create all possible states (2916 in total)
states = []
for comb in disease_combinations:
    for prob_comb in product(probabilities, repeat=3):
        state = {comb[i]: round(prob_comb[i], 1) for i in range(3)}  # Round probabilities to 1 decimal place
        states.append(state)
# print(states)

In [ ]:
# Initialize the Q-table with states as rows and symptoms as columns
q_table = pd.DataFrame(
    np.zeros((len(states), len(symptom))),
    index=[f"{list(state.keys())} : {list(state.values())}" for state in states],
    columns=symptom
)
# Display initial table
# q_table

In [ ]:
#Example State
# state = {'Dermatitis due to sun exposure': 0.1, 'Acne': 0.1, 'Eczema': 0.1}

In [ ]:
def policy(state, asked_symptoms, previous_asked_disease, reward):
    diseases = list(state.keys())
    # print(diseases)
    disease_probs = list(state.values())
    # print(disease_probs)


    if reward < 0 or previous_asked_disease == "":
        # Find the highest probability disease
        disease_index = np.argmax(disease_probs)
        # print(disease_index)
        chosen_disease = diseases[disease_index]
    else:
        # Continue with the previous disease if the reward was positive
        chosen_disease = previous_asked_disease

    # print(chosen_disease)
    # Get symptoms for the chosen disease sorted by probability
    symptoms = sorted(dataset[chosen_disease]['symptom'].items(), key=lambda x: x[1], reverse=True)
    # print(symptoms)

    # Select a symptom that hasn't been asked yet
    for symptom, _ in symptoms:
        if symptom not in asked_symptoms:
            return symptom, chosen_disease  # Return the symptom and the disease it was chosen from

    # If all symptoms have been asked, move to the next highest probability disease
    next_disease_index = np.argsort(disease_probs)[::-1]  # Sort diseases by descending probability
    # print(next_disease_index)
    for i in next_disease_index:
        symptoms = sorted(dataset[diseases[i]]['symptom'].items(), key=lambda x: x[1], reverse=True)
        for symptom, _ in symptoms:
            if symptom not in asked_symptoms:
                return symptom, diseases[i]

    # If all symptoms of all diseases have been asked (unlikely), return None
    return None, None

# state = {'Dermatitis due to sun exposure': 0.4, 'Acne': 0.6, 'Eczema': 0.3}
# Acne - ['Acne or pimples': 0.94,'Skin rash': 0.38,'Abnormal appearing skin': 0.21,'Skin moles': 0.2,'Skin swelling': 0.17,'Skin growth': 0.13,]
# Eczema - ['Skin rash': 0.78,'Itching of skin': 0.49,'Abnormal appearing skin': 0.45,'Skin dryness, peeling, scaliness, or roughness': 0.33,'Skin lesion': 0.29,'Cough': 0.24,]
# asked_symptoms = ['Acne or pimples','Skin rash','Abnormal appearing skin','Skin moles','Skin swelling','Skin growth']
# previous_asked_disease = "Eczema"
# choose_symptom = policy(state, asked_symptoms, previous_asked_disease, -0.5)
# print(choose_symptom)

In [ ]:
def symptom_relevance_factor(state, disease, symptom):
    similarities = []
    for other_disease in state.keys():
        if other_disease == disease:
            continue
        # print(other_disease)
        common_symptoms = set(dataset[disease]['symptom'].keys()).intersection(dataset[other_disease]['symptom'].keys())
        # print(common_symptoms)
        similarity = round(sum([dataset[disease]['symptom'][s] * dataset[other_disease]['symptom'][s] for s in common_symptoms]), 3)
        # print(similarity)
        if symptom in dataset[other_disease]['symptom']:
            similarities.append(round((dataset[other_disease]['symptom'][symptom] * similarity), 3))

    # print(similarities)
    srf_value = sum(similarities)
    # print(srf_value)
    # Normalize the probabilities between 0.1 and 0.9
    return round(max(0.1,min(srf_value,0.9)),2)
    # return srf_value / len(similarities) if similarities else 0.1  # Normalize SRF to range (0.1 to 0.9)

# state = {'Acne': 0.3, 'Eczema': 0.4, 'Dermatitis due to sun exposure': 0.5}
# disease = 'Acne'
# symptom = 'Skin lesion' # Symptom Skin lesion prasence and probability [Acne: not present, Eczema:0.29, Dermatitis: 0.51]
# Norm_srf_value = symptom_relevance_factor(state, disease, symptom)
# print(Norm_srf_value)

In [ ]:
#alpha beta value determination case study
# alpha = 0.0
# for i in range(9):
#   alpha += 0.1
#   symptom_prob = 0.0
#   print(f"alpha:{round(alpha,1)}")
#   for j in range(9):
#     symptom_prob += 0.1
#     disease_prob = 0.0
#     print(f"symptom_prob:{round(symptom_prob,1)}")
#     for k in range(9):
#       disease_prob += 0.1
#       print(f"For disease_prob:{round(disease_prob,1)}-value added:{round(alpha * symptom_prob * disease_prob,1)}")

In [ ]:
def update_disease_probabilities(state, symptom, answer, alpha=0.6, beta=0.6):# update alpha and beta value to 0.8.
    alpha = round(alpha,1)
    beta = round(beta,1)
    # print(alpha)
    # print(beta)
    new_state = state.copy()
    for disease in state.keys():
        symptom_prob = dataset[disease]['symptom'].get(symptom, 0)

        if answer == 'yes':
            if symptom in dataset[disease]['symptom']:  # Case 1
                new_state[disease] = state[disease] + alpha * symptom_prob * state[disease]
            else:  # Case 2
                srf = symptom_relevance_factor(state, disease, symptom)
                new_state[disease] = state[disease] - beta * srf * state[disease]
        elif answer == 'no':
            if symptom in dataset[disease]['symptom']:  # Case 3
                new_state[disease] = state[disease] - alpha * symptom_prob * state[disease]
            else:  # Case 4
                srf = symptom_relevance_factor(state, disease, symptom)
                new_state[disease] = state[disease] + beta * srf * state[disease]

        # Normalize the probabilities between 0.1 and 0.9
        new_state[disease] = round(max(0.1, min(new_state[disease], 0.9)), 1)

    return new_state

# state = {'Acne': 0.5, 'Eczema': 0.5, 'Dermatitis due to sun exposure': 0.5} #skin rash probability in [Acne:0.38, Eczema:0.78, Dermatitis: 0.45, Erythema: 0.92]
# symptom = 'Skin rash'
# answer = 'yes'
# # print(update_disease_probabilities(state, symptom, answer))
# alpha = 0.0
# # beta = 0.0
# for i in range(10):
#   alpha += 0.1
#   # beta += 0.1
#   print(update_disease_probabilities(state, symptom, answer, alpha))
#   # print(update_disease_probabilities(state, symptom, 'no',0.8,beta))

In [ ]:
#Deprecated Formula for confidence measurement
# import numpy as np
# def calculate_confidence(state):
#     max_prob = max(state.values())
#     entropy = -sum([p * np.log(p) for p in state.values() if p > 0])
#     print(round(entropy,4))
#     gamma = 0.5  # Adjustable confidence factor
#     return round(max_prob - gamma * entropy,3)

# def reward_function(new_state, old_state):
#     confidence_new = calculate_confidence(new_state)
#     print(confidence_new)
#     confidence_old = calculate_confidence(old_state)
#     print(confidence_old)
#     return round(confidence_new - confidence_old,3)

# old_state = {'Acne': 0.8, 'Eczema': 0.1, 'Dermatitis due to sun exposure': 0.1}
# new_state = {'Acne': 0.8, 'Eczema': 0.8, 'Dermatitis due to sun exposure': 0.8}
# reward = reward_function(new_state, old_state)
# reward

In [ ]:
#Main Formula for confidence measurement - standard deviation + difference of two highest probabilities.
def confidence(state):
    # Get the list of probabilities from the dictionary
    probabilities = list(state.values())

    # 1. Calculate standard deviation of probabilities
    std_dev = np.std(probabilities)

    # 2. Find the two highest probabilities and their difference
    sorted_probs = sorted(probabilities, reverse=True)
    top_two_diff = sorted_probs[0] - sorted_probs[1]  # Difference between the two highest probabilities

    # 3. Combine the two components into a single confidence score
    # You can tweak the weights (e.g., 0.7 and 0.3) to adjust the impact of each factor
    confidence_score = std_dev + top_two_diff

    return confidence_score

def calculate_reward(old_state, new_state):
    # Compute confidence values for both states
    old_confidence = confidence(old_state)
    # print(f"old_confidence:{round(old_confidence,4)}")
    new_confidence = confidence(new_state)
    # print(f"new_confidence:{round(new_confidence,4)}")

    # Reward is the change in confidence (new - old)
    reward = new_confidence - old_confidence
    return reward

# Example
# old_state = {'Acne': 0.7, 'Eczema': 0.2, 'Dermatitis due to sun exposure': 0.6}
# new_state = {'Acne': 0.9, 'Eczema': 0.3, 'Dermatitis due to sun exposure': 0.5}

# reward = calculate_reward(old_state, new_state)
# print(f"Confidence: {round(reward,4)}")

In [ ]:
# #Deprecated Formula for confidence measurement
# import numpy as np
# def confidence(state, epsilon=0.01):
#     # Get the list of probabilities from the dictionary
#     probabilities = list(state.values())

#     # Get the top 3 probabilities (since the formula works on a, b, c)
#     a, b, c = sorted(probabilities, reverse=True)[:3]

#     # Calculate the differences
#     diff_ab = abs(a - b)
#     diff_ac = abs(a - c)
#     diff_bc = abs(b - c)

#     # Numerator: sum of absolute differences multiplied by max(a, b, c)
#     numerator = (diff_ab + diff_ac + diff_bc) * max(a, b, c)

#     # Denominator: minimum of the differences plus epsilon
#     denominator = min(diff_ab, diff_ac, diff_bc) + epsilon

#     # Calculate the confidence score
#     confidence_score = numerator / denominator

#     return confidence_score

# def calculate_reward(old_state, new_state):
#     # Compute confidence values for both states
#     old_confidence = confidence(old_state)
#     print(f"old_confidence: {round(old_confidence, 4)}")
#     new_confidence = confidence(new_state)
#     print(f"new_confidence: {round(new_confidence, 4)}")

#     # Reward is the change in confidence (new - old)
#     reward = new_confidence - old_confidence
#     return reward

# # Example
# old_state = {'Acne': 0.7, 'Eczema': 0.2, 'Dermatitis due to sun exposure': 0.6}
# new_state = {'Acne': 0.9, 'Eczema': 0.3, 'Dermatitis due to sun exposure': 0.5}

# reward = calculate_reward(old_state, new_state)
# print(f"Reward: {round(reward, 4)}")


In [ ]:
def should_terminate(state, questions_asked, max_questions=8, threshold=0.6):
    # Check if the difference between top 2 probabilities exceeds the threshold
    probs = sorted(state.values(), reverse=True)
    if len(probs) > 1 and (probs[0] - probs[1] >= threshold):
        return True
    # Terminate if the max question limit is reached
    if questions_asked >= max_questions:
        return True
    return False


In [ ]:
def train_q_table(q_table, epochs, alpha=0.1, gamma=0.9):
    for epoch in range(epochs):
        # Initialize a random starting state
        current_state = states[np.random.randint(0, len(states))]
        #print(current_state) - example:{'Acne': 0.2, 'Erythema multiforme': 0.7, 'Dermatitis due to sun exposure': 0.4}
        # Extract diseases and their probabilities, and sort for consistency
        diseases = list(current_state.keys())
        probabilities = list(current_state.values())
        # Format the state index as the desired string
        state_index = f"{diseases} : {probabilities}"
        #print(state_index) - ['Acne', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.2, 0.7, 0.4]
        asked_symptoms = []
        previous_asked_disease = ""
        reward = 0
        questions_asked = 0

        while not should_terminate(current_state, questions_asked):
            print(f"index:{state_index}, current state:{current_state}")

            # Choose action (symptom) using the policy
            symptom, selected_disease = policy(current_state, asked_symptoms, previous_asked_disease, reward)
            if symptom is None:
                break  # No more symptoms to ask

            # Simulate patient's answer (random for now, replace with actual patient interaction)
            patient_answer = np.random.choice(['yes', 'no'])

            # Store previous state
            old_state = current_state.copy()

            # Update the state based on patient's answer
            current_state = update_disease_probabilities(current_state, symptom, patient_answer)

            # Measure reward
            reward = calculate_reward(current_state, old_state)

            # best_future_q = q_table.iloc[get_state_index(state)].max()
            # q_table.iloc[state_index, action_index] = round(q_table.iloc[state_index, action_index] + alpha * (reward + gamma * best_future_q - q_table.iloc[state_index, action_index]), 3)

            # Update Q-table
            # q_table.loc[old_state, symptom] = round(q_table.loc[old_state, symptom] + alpha * (reward + gamma * q_table.loc[old_state].max()- q_table.loc[old_state, symptom]),3)

            # Update Q-table
            q_table.loc[state_index, symptom] = round((1 - alpha) * q_table.loc[state_index, symptom] + alpha * (reward + gamma * q_table.loc[state_index].max()),3)


            # Update asked symptoms and previous disease
            asked_symptoms.append(symptom)
            previous_asked_disease = selected_disease
            questions_asked += 1

    return q_table

# Train the Q-table with the defined parameters
trained_q_table = train_q_table(q_table, 100)

index:['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.7, 0.5, 0.2], current state:{'Eczema': 0.7, 'Erythema multiforme': 0.5, 'Dermatitis due to sun exposure': 0.2}
index:['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.7, 0.5, 0.2], current state:{'Eczema': 0.9, 'Erythema multiforme': 0.8, 'Dermatitis due to sun exposure': 0.3}
index:['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.7, 0.5, 0.2], current state:{'Eczema': 0.6, 'Erythema multiforme': 0.6, 'Dermatitis due to sun exposure': 0.4}
index:['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.7, 0.5, 0.2], current state:{'Eczema': 0.4, 'Erythema multiforme': 0.9, 'Dermatitis due to sun exposure': 0.2}
index:['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.7, 0.5, 0.2], current state:{'Eczema': 0.6, 'Erythema multiforme': 0.5, 'Dermatitis due to sun exposure': 0.3}
index:['Eczema', 'Erythema multiforme', 'Dermatiti

In [ ]:
trained_q_table

,Abnormal appearing skin,Acne or pimples,Coryza,Cough,Fever,Irregular appearing scalp,Itching of skin,"Skin dryness, peeling, scaliness, or roughness",Skin growth,Skin lesion,Skin moles,Skin rash,Skin swelling,Vomiting
"['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.1, 0.1]",0.001,0.0,0.001,0.0,0.015,0.0,0.001,0.0,0.0,0.001,0.0,-0.015,0.0,0.001
"['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.1, 0.2]",0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000
"['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.1, 0.3]",0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000
"['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.1, 0.4]",0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000
"['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.1, 0.5]",0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.9, 0.9, 0.5]",0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000
"['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.9, 0.9, 0.6]",0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000
"['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.9, 0.9, 0.7]",0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000
"['Eczema', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.9, 0.9, 0.8]",0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000


In [ ]:
# old_state = "['Acne', 'Erythema multiforme', 'Dermatitis due to sun exposure'] : [0.2, 0.4, 0.4]"
# symptom = "Abnormal appearing skin"
# print(trained_q_table.loc[old_state].max())
# print(trained_q_table.loc[old_state, symptom])

In [ ]:
def interact_with_model(q_table, initial_state):
    current_state = initial_state
    diseases = list(current_state.keys())
    probabilities = list(current_state.values())
    state_index = f"{diseases} : {probabilities}"
    asked_symptoms = []
    previous_asked_disease = ""
    questions_asked = 0

    print(f"Initial State: {current_state}")

    while not should_terminate(current_state, questions_asked):
        # Choose action (symptom) using the trained Q-table
        print(f"state_index:{state_index}")
        q_values = q_table.loc[state_index]
        print(f"q_values:{q_values}")

        # Sort symptoms by Q-value in decreasing order
        sorted_symptoms = q_values.sort_values(ascending=False).index.tolist()
        print(f"sorted_symptoms:{sorted_symptoms}")
        # Select the next symptom that hasn't been asked
        for symptom in sorted_symptoms:
            if symptom not in asked_symptoms:
                selected_symptom = symptom
                asked_symptoms.append(selected_symptom)
                break

        print(f"selected_symptom:{selected_symptom}")

        # Perform action (ask the user the selected symptom)
        answer = input(f"Do you have {selected_symptom}? (yes/no): ").strip().lower()
        # Ask the question
        # print(f"Question {questions_asked + 1}: Do you have {symptom}? (yes/no)")
        # patient_answer = input().lower()

        # Update the state based on patient's answer
        old_state = current_state.copy()
        current_state = update_disease_probabilities(old_state, selected_symptom, answer)
        print(f"new state:{current_state}")
        # current_state = {d: round(p, 1) for d, p in current_state.items()}  # Round to 1 decimal point

        # print(f"Updated Disease Probabilities: {current_state}")

        # Update variables
        questions_asked += 1
        diseases = list(current_state.keys())
        probabilities = list(current_state.values())
        state_index = f"{diseases} : {probabilities}"


    print("Interaction complete.")



# Define an initial state (from user or pre-set)
initial_state = {'Acne': 0.1, 'Eczema': 0.4, 'Erythema multiforme': 0.6}
# Interact with the model using the trained Q-table
interact_with_model(trained_q_table, initial_state)


Initial State: {'Acne': 0.1, 'Eczema': 0.4, 'Erythema multiforme': 0.6}
state_index:['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.4, 0.6]
q_values:Abnormal appearing skin                          -0.023
Acne or pimples                                   0.000
Coryza                                            0.017
Cough                                             0.017
Fever                                             0.002
Irregular appearing scalp                         0.000
Itching of skin                                  -0.002
Skin dryness, peeling, scaliness, or roughness   -0.013
Skin growth                                       0.000
Skin lesion                                       0.005
Skin moles                                        0.000
Skin rash                                         0.022
Skin swelling                                     0.000
Vomiting                                          0.000
Name: ['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.4, 0.

In [ ]:
# value = q_table.loc["['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.1, 0.1]", symptom]
# value

,"['Acne', 'Eczema', 'Erythema multiforme'] : [0.1, 0.1, 0.1]"
Abnormal appearing skin,0.0
Acne or pimples,0.0
Coryza,0.0
Cough,0.0
Fever,0.0
Irregular appearing scalp,0.0
Itching of skin,0.0
"Skin dryness, peeling, scaliness, or roughness",0.0
Skin growth,0.0
Skin lesion,0.0


In [ ]:
# # Extract the list of diseases and their indices
# disease_index_map = {disease: details['index'] for disease, details in dataset.items()}
# # Function to generate the index for a given state
# def generate_state_index(state):
#     # Do not sort diseases, maintain the order as provided in the input
#     disease_indices = [disease_index_map[disease] for disease in state.keys()]

#     # Map the probabilities to indices (0 for 0.1, 1 for 0.2, ..., 8 for 0.9)
#     probability_indices = [(int((state[disease] * 10) - 1)) for disease in state.keys()]

#     # Calculate the unique index
#     index = 0
#     index += probability_indices[0] * (9**2)  # First probability
#     index += probability_indices[1] * 9       # Second probability
#     index += probability_indices[2]           # Third probability

#     # Adjust for the disease combination (based on the current disease order)
#     comb_index = disease_combinations.index(tuple(state.keys()))
#     index += comb_index * (9**3)  # Adjust index based on disease combination

#     return index

# # Example usage
# state1 = {'Acne': 0.8, 'Erythema multiforme': 0.1, 'Dermatitis due to sun exposure': 0.9}  # Should return [1, 2, 4]
# state2 = {'Acne': 0.1, 'Eczema': 0.1, 'Erythema multiforme': 0.1}  # Example for a different state

# print(f"Disease indices for {state1}: {[disease_index_map[disease] for disease in state1.keys()]}")
# print(f"Index for {state1}: {generate_state_index(state1)}")

# print(f"Disease indices for {state2}: {[disease_index_map[disease] for disease in state2.keys()]}")
# print(f"Index for {state2}: {generate_state_index(state2)}")